<a href="https://colab.research.google.com/github/Kzis/my-saprk/blob/main/ttba_spark_lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.1/spark-2.4.1-bin-hadoop2.7.tgz
!tar xf spark-2.4.1-bin-hadoop2.7.tgz
!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.1-bin-hadoop2.7"

In [3]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder \
   .appName("Neural Network Model") \
   .config("spark.executor.memory", "3gb") \
   .getOrCreate()
   
sc = spark.sparkContext

In [6]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

## Lab 1

Create a local collection of colors first, a Python list


In [7]:
colors = ['white','green','yellow','red','brown','pink'] 

Distribute the local collection to be an RDD. 


In [8]:
color_rdd = sc.parallelize(colors)


Apply map function on that RDD to get another RDD containing color, length tuples.

In [9]:
keyval_rdd = color_rdd.map(lambda x:(x,len(x)))


Convert the last RDD to a DataFrame 


In [10]:
color_df = keyval_rdd.toDF(['color','length']) 
color_df

DataFrame[color: string, length: bigint]

In [11]:
color_df.printSchema()

root
 |-- color: string (nullable = true)
 |-- length: long (nullable = true)



In [12]:
color_df.count() 


6

Look at the table contents. You can limit displayed rows by passing parameter to show.

In [13]:
color_df.show(2)


+-----+------+
|color|length|
+-----+------+
|white|     5|
|green|     5|
+-----+------+
only showing top 2 rows



In [14]:
color_df.columns


['color', 'length']

In [15]:
color_df.drop('length').show() 


+------+
| color|
+------+
| white|
| green|
|yellow|
|   red|
| brown|
|  pink|
+------+



In [16]:
color_df.filter(color_df.length.between(4,5))\
.select(color_df.color.alias("mid_length")).show()


+----------+
|mid_length|
+----------+
|     white|
|     green|
|     brown|
|      pink|
+----------+



In [17]:
color_df.filter(color_df.length > 4) .filter(color_df[0]!="white").show()


+------+------+
| color|length|
+------+------+
| green|     5|
|yellow|     6|
| brown|     5|
+------+------+



In [18]:
color_df.filter((color_df.length > 4)&(color_df[0]!="white")).show()


+------+------+
| color|length|
+------+------+
| green|     5|
|yellow|     6|
| brown|     5|
+------+------+



In [19]:
color_df.filter(color_df['length']>=4).sort("length", 'color',ascending=False).show()

+------+------+
| color|length|
+------+------+
|yellow|     6|
| white|     5|
| green|     5|
| brown|     5|
|  pink|     4|
+------+------+



In [20]:
color_df.orderBy('length','color').take(4)


[Row(color='red', length=3),
 Row(color='pink', length=4),
 Row(color='brown', length=5),
 Row(color='green', length=5)]

In [21]:
color_df.sort(color_df.length.desc(),\
color_df.color.asc()).show()


+------+------+
| color|length|
+------+------+
|yellow|     6|
| brown|     5|
| green|     5|
| white|     5|
|  pink|     4|
|   red|     3|
+------+------+



In [22]:
color_df.groupBy('length').count().show()


+------+-----+
|length|count|
+------+-----+
|     6|    1|
|     5|    3|
|     3|    1|
|     4|    1|
+------+-----+



In [23]:
color_df.describe().show()


+-------+------+------------------+
|summary| color|            length|
+-------+------+------------------+
|  count|     6|                 6|
|   mean|  null| 4.666666666666667|
| stddev|  null|1.0327955589886444|
|    min| brown|                 3|
|    max|yellow|                 6|
+-------+------+------------------+

